In [1]:
%pip install torch torchvision torchaudio

In [2]:
%pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-$(torch --version | awk -F '[: ]+' '{print $3}')+cu$(nvidia-smi --query-gpu=driver_version --format=csv,noheader | awk -F '.' '{print $1$2}')/torch_stable.html


/bin/bash: line 1: torch: command not found
/bin/bash: line 1: nvidia-smi: command not found
Looking in links: https://data.pyg.org/whl/torch-+cu/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.4 MB/s eta 0:00:00
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=537340 sha256=6ce006d0e026e94d0c4fbc0d1fa4bd98cce1ec5cdec291d2b037c2cc4896e607
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c83635685690532480408

In [3]:
# Dataset Connection
import torch
from torch_geometric.datasets import Planetoid
dataset_name = 'Cora'
dataset = Planetoid(root='data/' + dataset_name, name=dataset_name)

Processing...
Done!


In [35]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [148]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import sys
from scipy import sparse
import tensorflow as tf
from tensorflow.keras import layers

# Data Loading
data = dataset[0]
objects = []
#Node features
node_features = data.x
objects.append(node_features)
#Node labels
node_labels = data.y
objects.append(node_labels)
#Adjacency Matrix (Graph adjacency information)
graph = data.edge_index
#objects.append(graph)

node_features, node_labels = tuple(objects)

#Data Preprocessing
features = sparse.csr_matrix(node_features)
test_idx_reorder = data.test_mask.nonzero(as_tuple=False).view(-1).numpy()
test_idx_range = np.sort(test_idx_reorder)

features[test_idx_reorder, :] = features[test_idx_range, :]

#row features normalization
row_sum = np.array(features.sum(1))
row_inv = np.power(row_sum, -1).flatten()
row_inv[np.isinf(row_inv)] = 0.
row_mat_inv = sp.diags(row_inv)
features = row_mat_inv.dot(features)
features = features.todense()


nb_nodes = features.shape[0]
ft_size = features.shape[1]
batch_size = 1
hid_units = [8] #No of output size
n_heads = [8, 1]
nonlinearity = tf.nn.elu
residual = False

#Defining Input layers
# feature_input_shape = (batch_size, nb_nodes, ft_size)
# feature_input= layers.Input(shape=feature_input_shape[1:])

# bias_input = layers.Input(shape=(batch_size, nb_nodes, nb_nodes))

feature_input = tf.keras.Input(shape=(nb_nodes, ft_size), batch_size=batch_size)
bias_input = tf.keras.Input(shape=(nb_nodes, nb_nodes), batch_size=batch_size)

#Defining dropout rates
ffd_drop = 0.0
attn_drop = 0.0

seq = feature_input


conv1d = layers.Conv1D #layers.Conv1D

# Attention Mechanism
def attn_head(seq, bias_mat, out_sz, activation, in_drop=0.0, coef_drop=0.0, residual=False):
  with tf.name_scope('my_attn'):
          # Apply dropout if needed
          if in_drop != 0.0:
              seq = tf.nn.dropout(seq, 1.0 - in_drop)

          seq_fts = layers.Conv1D(filters=out_sz, kernel_size=1, use_bias=False)(seq)

          # Self-attention mechanism
          f_1 = layers.Conv1D(filters=1, kernel_size=1)(seq_fts)
          f_2 = layers.Conv1D(filters=1, kernel_size=1)(seq_fts)

          f_2_transposed = layers.Lambda(lambda x: tf.transpose(x, perm=[0, 2, 1]))(f_2)
          logits = layers.Add()([f_1, f_2_transposed])

          leaky_relu = layers.LeakyReLU(alpha=0.2)
          logits_with_bias = layers.Add()([logits, bias_mat])
          coefs = layers.Activation('softmax')(leaky_relu(logits_with_bias))
          coefs_reshaped = layers.Reshape((2708, 2708))(coefs)




          if coef_drop != 0.0:
              coefs = tf.nn.dropout(coefs, 1.0 - coef_drop)

          # Matrix Multiplication
          vals = layers.Dot(axes=[1,1])([coefs_reshaped, seq_fts])
          #ret = vals

          # add_layer = tf.keras.layers.Add()
          # ret = add_layer([vals, bias_mat])

          # Residual connection using tf.cond
          if residual:
              ret = tf.cond(
                  tf.shape(seq)[-1] != tf.shape(ret)[-1],  # Condition
                  lambda: ret + layers.Conv1D(tf.shape(ret)[-1], 1)(seq),  # If true
                  lambda: ret + seq  # If false
              )

          if isinstance(activation, str):
            ret = layers.Activation(activation)(vals)  # If activation is a string name
          else:
            activation_layer = CustomActivation()
            ret = activation_layer(vals)

          return ret, coefs

class CustomActivation(layers.Layer):
    def call(self, inputs):
        return tf.nn.leaky_relu(inputs, alpha=0.2)

output, attention_weights  = attn_head(seq, bias_mat=bias_input,
              out_sz=hid_units[0], activation=nonlinearity,
              in_drop=ffd_drop, coef_drop=attn_drop, residual=False)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [149]:
model = tf.keras.Model(inputs=[feature_input, bias_input], outputs=[output, attention_weights])
feature_input_pred = tf.random.normal(shape=(batch_size, nb_nodes, ft_size))
bias_input_pred = tf.random.normal(shape=(batch_size, nb_nodes, nb_nodes))
result, attention_weights  = model.predict([feature_input_pred, bias_input_pred]) #result presents the posibilites of nodes belonging to different classes
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
[[[-5.93792181e-03  1.33593967e-02  7.56769581e-03 ...  1.49006033e-02
   -2.22952847e-04 -6.50201458e-03]
  [ 3.86376167e-04 -1.23089720e-02  3.65208648e-02 ...  9.33311656e-02
    6.25395998e-02 -1.31424805e-02]
  [ 3.31265032e-02  2.44479589e-02  3.03144865e-02 ...  4.48021181e-02
    2.53930734e-03  3.45284306e-02]
  ...
  [ 2.98298895e-04 -5.88089507e-03  1.42798899e-02 ...  4.23757397e-02
    2.99689248e-02 -6.09180797e-03]
  [-9.39240563e-05 -1.31790508e-02  3.49502563e-02 ...  9.49009731e-02
    6.11336567e-02 -1.29420180e-02]
  [-2.45917327e-04 -1.37911802e-02  3.31422053e-02 ...  9.76806954e-02
    6.19298741e-02 -1.21397590e-02]]]


In [125]:
#Attention weights from one node to all other nodes
node_index = 40  # Index of the node you want to inspect
print(f"Attention weights for node {node_index} to its neighbors:")
print(attention_weights[0, node_index, :])

Attention weights for node 40 to its neighbors:
[0.00016312 0.00020975 0.00015461 ... 0.00014715 0.00173849 0.00021143]


In [150]:
# Attention weights between two nodes
edge_example = edge_index[:, np.where(edge_index[0]==40)[0]]
print(edge_example)

tensor([[  40,   40,   40],
        [ 507,  866, 1364]])


In [151]:
node_index_1 = edge_example[0][0]
node_index_2 = edge_example[1][0]
print(f"Attention weights for node {node_index_1} to {node_index_2}")
print(attention_weights[0, node_index_1, node_index_2 ])


Attention weights for node 40 to 507
5.1700645e-05


In [153]:
edge_index = dataset[0].edge_index
node_features = dataset[0].x

# Create a NetworkX graph from the edge index
G = nx.Graph()

# Add edges to the graph from the edge_index
for i in range(edge_index.shape[1]):
    src = edge_index[0, i].item()  # Source node
    dst = edge_index[1, i].item()  # Destination node
    G.add_edge(src, dst)

# Get the adjacency matrix as a sparse matrix
adjacency_matrix = nx.adjacency_matrix(G)

# Print the shape of the adjacency matrix
print(adjacency_matrix.shape)

(2708, 2708)


In [127]:
print(graph)

tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])
